# Weekend shootings and homicides?

How many people were shot each weekend?  How many people were killed in a homicide?  Which weekends had the most of both types of violence?

For this analysis, we define a weekend as starting on Friday at 3 p.m. and ending on Monday at 6 a.m.

## Load data from Newsroom DB

In [22]:
import os
import requests

# Some constants
NEWSROOMDB_URL = os.environ['NEWSROOMDB_URL']

# Utilities for loading data from NewsroomDB

def get_table_url(table_name, base_url=NEWSROOMDB_URL):
    return '{}table/json/{}'.format(base_url, table_name)

def get_table_data(table_name):
    url = get_table_url(table_name)
    
    try:
        r = requests.get(url)
        return r.json()
    except:
        print("Request failed. Probably because the response is huge.  We should fix this.")
        return get_table_data(table_name)

shooting_victims_raw = get_table_data('shootings')
print("Loaded {} shooting victims".format(len(shooting_victims_raw)))

Loaded 12675 shooting victims


In [23]:
import agate
from datetime import datetime, timedelta

# Load raw data into an Agate table

# Agate tries to parse the date and time automatically. It parses the time incorrectly
# as MM:SS instead of HH:MM. We ultimately need a timestamp, which is easily
# parsed by concatenating the date and time, so disable the initial
# auto-parsing of these fields.
column_types = {
    'Date': agate.Text(),
    'Time': agate.Text(),
}
shooting_victims = agate.Table.from_object(shooting_victims_raw, column_types=column_types)

# Calculate a timestamp from the Date and Time columns

def get_timestamp(row, date_col='Date', time_col='Time'):    
    if not row[date_col] or not row[time_col]:
        return None
    
    try:
        timestamp = datetime.strptime("{} {}".format(row[date_col], row[time_col]), "%Y-%m-%d %H:%M")
    except ValueError:
        timestamp = datetime.strptime("{} {}".format(row[date_col], row[time_col]), "%Y-%m-%d %H:%M:%S")
    
    # HACK: There are some bad dates in the data.  Based on visual inspection,
    # we can fix the dates using a couple of rules
    year = timestamp.year
    if year < 20:
        year += 2000
        new_timestamp = timestamp.replace(year=year)
        print("Bad year date in row with id {}. Changing {} to {}.".format(
            row['_id'], timestamp.strftime("%Y-%m-%d"), new_timestamp.strftime("%Y-%m-%d")))
        timestamp = new_timestamp
    elif year == 216:
        new_timestamp = timestamp.replace(year=2016)
        print("Bad year date in row with id {}. Changing {} to {}.".format(
            row['_id'], timestamp.strftime("%Y-%m-%d"), new_timestamp.strftime("%Y-%m-%d")))
        timestamp = new_timestamp
    
    return timestamp

shooting_victims = shooting_victims.compute([
    ('timestamp', agate.Formula(agate.DateTime(), get_timestamp))
])

shooting_victims = shooting_victims.where(lambda row: row['timestamp'] is not None)


Bad year date in row with id 53b6a23cdb573e256fa3b14c. Changing 0014-07-22 to 2014-07-22.
Bad year date in row with id 5707a7f8389ce82a2bd86eed. Changing 0016-04-02 to 2016-04-02.
Bad year date in row with id 57419172db573e070ae4c812. Changing 0016-05-20 to 2016-05-20.
Bad year date in row with id 5756b0e7389ce82a2bd8717e. Changing 0216-06-07 to 2016-06-07.
Bad year date in row with id 575fde1c389ce82a2bd871c7. Changing 0016-06-13 to 2016-06-13.
Bad year date in row with id 576a820f389ce82a2a5b0cc9. Changing 0216-06-21 to 2016-06-21.


## Filter to only weekend shootings

In [24]:
def is_weekend(timestamp):
    """Does the timestamp fall between Friday 3 p.m. and Monday 6 a.m."""
    if not timestamp:
        return False
    
    day_of_week = timestamp.weekday()
    
    if day_of_week > 0 and day_of_week < 4:
        return False
    
    if day_of_week == 4:
        # Friday
        
        # Same day, 3 p.m.
        start = datetime(timestamp.year, timestamp.month, timestamp.day, 15)
        
        return timestamp >= start
    
    if day_of_week == 0:
        # Monday
        
        # Same day, 6 a.m.
        end = datetime(timestamp.year, timestamp.month, timestamp.day, 6)
        
        return timestamp < end
        
    return True

weekend_shootings = shooting_victims.where(lambda row: is_weekend(row['timestamp']))
print("There are {0} weekend shooting victims".format(len(weekend_shootings.rows)))

There are 6230 weekend shooting victims


In [25]:
from datetime import datetime
import time

# Utility functions for calculating weekend start and end dates/times for a given 

def clone_datetime(d):
    """Make a copy of a datetime object"""
    # HACK: Is there a better way to do this?  Why isn't there an obvious clone method?
    return datetime.fromtimestamp(time.mktime(d.timetuple()))

# The following methods only work for timestamps that fall within a weekend

def weekend_start(timestamp):
    days_from_friday = timestamp.weekday() - 4
    
    if days_from_friday < 0:
        days_from_friday += 1
        days_from_friday *= -1
        
    friday_delta = timedelta(days=(-1 * days_from_friday))
    
    start = clone_datetime(timestamp)
    
    start += friday_delta
    start = start.replace(hour=15, minute=0, second=0)
    
    return start

def weekend_end(timestamp):
    days_to_monday = 0 - timestamp.weekday()
    
    if days_to_monday < 0:
        days_to_monday += 7
        
    monday_delta = timedelta(days=days_to_monday)
    
    end = clone_datetime(timestamp)
    
    end += monday_delta
    end = end.replace(hour=6, minute=0, second=0)
    
    return end

def get_weekend_start(row):
    return weekend_start(row['timestamp']).date()

In [26]:
# Add weekend start and end dates to each row so we can
# group by on them later.  Cecilia took a different approach,
# calculating the weekends first and iterating through them
# and finding matching shootings for each weekend.
weekend_shootings_with_start_end = weekend_shootings.compute([
    ('weekend_start', agate.Formula(agate.Date(), get_weekend_start)),
    ('weekend_end', agate.Formula(agate.Date(), lambda row: weekend_end(row['timestamp']).date()))
])

In [27]:
# Aggregate the shooting victims by weekend
shooting_victims_by_weekend = weekend_shootings_with_start_end.group_by(
    lambda row: row['weekend_start'].strftime("%Y-%m-%d") + " to " +  row['weekend_end'].strftime("%Y-%m-%d"))

shooting_victims_weekend_counts = shooting_victims_by_weekend.aggregate([
    ('count', agate.Count())
])

shooting_victims_weekend_counts.order_by('count', reverse=True).print_table(max_column_width=40, max_rows=None)

|---------------------------+--------|
|  group                    | count  |
|---------------------------+--------|
|  2014-07-04 to 2014-07-07 |    67  |
|  2012-06-08 to 2012-06-11 |    59  |
|  2016-06-17 to 2016-06-20 |    59  |
|  2015-07-03 to 2015-07-06 |    57  |
|  2016-06-24 to 2016-06-27 |    56  |
|  2015-09-25 to 2015-09-28 |    56  |
|  2012-03-16 to 2012-03-19 |    55  |
|  2015-09-18 to 2015-09-21 |    54  |
|  2013-06-14 to 2013-06-17 |    53  |
|  2016-05-06 to 2016-05-09 |    51  |
|  2016-05-27 to 2016-05-30 |    50  |
|  2015-08-07 to 2015-08-10 |    48  |
|  2014-07-18 to 2014-07-21 |    46  |
|  2014-08-22 to 2014-08-25 |    46  |
|  2015-05-15 to 2015-05-18 |    46  |
|  2014-04-18 to 2014-04-21 |    45  |
|  2016-06-10 to 2016-06-13 |    45  |
|  2012-06-15 to 2012-06-18 |    44  |
|  2012-08-24 to 2012-08-27 |    44  |
|  2016-04-22 to 2016-04-25 |    44  |
|  2016-05-20 to 2016-05-23 |    44  |
|  2012-05-25 to 2012-05-28 |    43  |
|  2014-09-26 to 2014-09-

## Do the same thing for Homicides

In [28]:
homicides_raw = get_table_data('homicides')

homicide_column_types = {
    'Occ Date': agate.Text(),
    'Occ Time': agate.Text(),
}
homicides = agate.Table.from_object(homicides_raw, column_types=homicide_column_types)
homicides = homicides.compute([
    ('timestamp', agate.Formula(agate.DateTime(), lambda row: get_timestamp(row, date_col='Occ Date', time_col='Occ Time')))
])

weekend_homicides = homicides.where(lambda row: is_weekend(row['timestamp']))
weekend_homicides_with_start_end = weekend_homicides.compute([
    ('weekend_start', agate.Formula(agate.Date(), get_weekend_start)),
    ('weekend_end', agate.Formula(agate.Date(), lambda row: weekend_end(row['timestamp']).date()))
])

In [29]:
homicides_by_weekend = weekend_homicides_with_start_end.group_by(
    lambda row: row['weekend_start'].strftime("%Y-%m-%d") + " to " +  row['weekend_end'].strftime("%Y-%m-%d"))

weekend_homicide_counts = homicides_by_weekend.aggregate([
    ('count', agate.Count())
])

weekend_homicide_counts.order_by('count', reverse=True).print_table(max_column_width=40, max_rows=None)

|---------------------------+--------|
|  group                    | count  |
|---------------------------+--------|
|  2016-06-17 to 2016-06-20 |    13  |
|  2013-06-14 to 2013-06-17 |    11  |
|  2014-07-04 to 2014-07-07 |    11  |
|  2013-08-16 to 2013-08-19 |    10  |
|  2015-09-18 to 2015-09-21 |    10  |
|  2014-09-05 to 2014-09-08 |     9  |
|  2016-05-06 to 2016-05-09 |     9  |
|  2014-04-18 to 2014-04-21 |     9  |
|  2014-05-30 to 2014-06-02 |     9  |
|  2015-05-22 to 2015-05-25 |     9  |
|  2015-07-03 to 2015-07-06 |     9  |
|  2016-01-29 to 2016-02-01 |     9  |
|  2014-08-29 to 2014-09-01 |     8  |
|  2014-11-07 to 2014-11-10 |     8  |
|  2016-05-13 to 2016-05-16 |     8  |
|  2016-06-03 to 2016-06-06 |     8  |
|  2014-09-26 to 2014-09-29 |     8  |
|  2015-05-29 to 2015-06-01 |     8  |
|  2015-07-10 to 2015-07-13 |     8  |
|  2016-06-24 to 2016-06-27 |     8  |
|  2013-01-25 to 2013-01-28 |     7  |
|  2016-06-10 to 2016-06-13 |     7  |
|  2015-10-23 to 2015-10-

## What about both?

In [30]:
import re

# First off, we need to avoid double-counting homicides and shootings
def is_homicide(row):
    if not row['UCR']:
        return False
    
    if re.match(r'0{0,1}110', row['UCR']):
        return True
    
    return False

non_homicide_weekend_shootings = weekend_shootings_with_start_end.where(lambda row: not is_homicide(row))
print("There are {0} non-homicide weekend shootings".format(len(non_homicide_weekend_shootings.rows)))

non_homicide_shooting_victims_by_weekend = non_homicide_weekend_shootings.group_by(
    lambda row: row['weekend_start'].strftime("%Y-%m-%d") + " to " +  row['weekend_end'].strftime("%Y-%m-%d"))

non_homicide_shooting_victims_weekend_counts = non_homicide_shooting_victims_by_weekend.aggregate([
    ('count', agate.Count())
])

There are 5322 non-homicide weekend shootings


In [31]:
def none_to_zero(x):
    if x is None:
        return 0
    
    return x

shooting_victims_and_homicides = non_homicide_shooting_victims_weekend_counts.join(weekend_homicide_counts, 'group')
shooting_victims_and_homicides = shooting_victims_and_homicides.compute([
    ('total', agate.Formula(agate.Number(), lambda row: row['count'] + none_to_zero(row['count2']))),
])
shooting_victims_and_homicides.order_by('total', reverse=True).print_table(max_column_width=40, max_rows=None)

|---------------------------+-------+--------+--------|
|  group                    | count | count2 | total  |
|---------------------------+-------+--------+--------|
|  2014-07-04 to 2014-07-07 |    56 |     11 |    67  |
|  2015-07-03 to 2015-07-06 |    51 |      9 |    60  |
|  2016-06-17 to 2016-06-20 |    47 |     13 |    60  |
|  2013-06-14 to 2013-06-17 |    47 |     11 |    58  |
|  2016-06-24 to 2016-06-27 |    48 |      8 |    56  |
|  2015-09-25 to 2015-09-28 |    52 |      4 |    56  |
|  2015-09-18 to 2015-09-21 |    45 |     10 |    55  |
|  2016-05-06 to 2016-05-09 |    43 |      9 |    52  |
|  2016-05-27 to 2016-05-30 |    46 |      5 |    51  |
|  2012-06-08 to 2012-06-11 |    50 |        |    50  |
|  2015-08-07 to 2015-08-10 |    45 |      3 |    48  |
|  2014-09-26 to 2014-09-29 |    39 |      8 |    47  |
|  2015-05-15 to 2015-05-18 |    45 |      2 |    47  |
|  2012-03-16 to 2012-03-19 |    46 |        |    46  |
|  2014-07-18 to 2014-07-21 |    41 |      5 |  